##Performing Classification on the dataset
####Predicting the size of the package of the shipped items 
####Using a Stochastic Gradient Descent Classifier 


In [ ]:
!pip install pycaret 
import pandas as pd 
from sklearn.linear_model import SGDClassifier 
from sklearn.preprocessing import StandardScaler 
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt

     |████████████████████████████████| 264 kB 5.2 MB/s 
     |████████████████████████████████| 271 kB 41.6 MB/s 
     |████████████████████████████████| 80 kB 8.7 MB/s 
     |████████████████████████████████| 14.6 MB 111 kB/s 
     |████████████████████████████████| 248 kB 64.8 MB/s 
     |████████████████████████████████| 56 kB 4.3 MB/s 
     |████████████████████████████████| 1.3 MB 43.6 MB/s 
     |████████████████████████████████| 6.8 MB 39.7 MB/s 
     |████████████████████████████████| 167 kB 29.7 MB/s 
     |████████████████████████████████| 2.0 MB 27.1 MB/s 
     |████████████████████████████████| 1.7 MB 51.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 111 kB 45.2 MB/s 
     |████████████████████████████████| 675 kB 55.9 MB/s 
     |████████████████████████████████| 62 kB 699 kB/s 
     |████████████████████

In [ ]:
#loading the dataset into a pandas dataframe 
df = pd.read_excel("/content/sales_data_sample.xlsx",usecols=['QUANTITYORDERED','PRICEEACH','SALES','STATUS','PRODUCTLINE','DEALSIZE'])
print(df.shape)
df.head()

(2823, 6)


,QUANTITYORDERED,PRICEEACH,SALES,STATUS,PRODUCTLINE,DEALSIZE
0,30,95.70,2871.00,Shipped,Motorcycles,Small
1,34,81.35,2765.90,Shipped,Motorcycles,Small
2,41,94.74,3884.34,Shipped,Motorcycles,Medium
3,45,83.26,3746.70,Shipped,Motorcycles,Medium
4,49,100.00,5205.27,Shipped,Motorcycles,Medium


In [ ]:
is_Shipped = df['STATUS'] == 'Shipped'
filtered_df = df[is_Shipped]
filtered_df = filtered_df.reset_index()
print(filtered_df.shape)
filtered_df.head()

(2617, 7)


,index,QUANTITYORDERED,PRICEEACH,SALES,STATUS,PRODUCTLINE,DEALSIZE
0,0,30,95.70,2871.00,Shipped,Motorcycles,Small
1,1,34,81.35,2765.90,Shipped,Motorcycles,Small
2,2,41,94.74,3884.34,Shipped,Motorcycles,Medium
3,3,45,83.26,3746.70,Shipped,Motorcycles,Medium
4,4,49,100.00,5205.27,Shipped,Motorcycles,Medium


In [ ]:
#QUANTITYORDERED, PRICEEACH, SALES, PRODUCTLINE (ENCODED) as input 
#DEALSIZE as output 
#let's find the unique values of the categorical column PRODUCTLINE and covert them to numerical representations by using label encoding 
print(filtered_df.PRODUCTLINE.unique())
filtered_df['PRODUCTLINE'] = filtered_df['PRODUCTLINE'].astype('category')
filtered_df['PRODUCTLINE_ENCODED'] = filtered_df['PRODUCTLINE'].cat.codes
filtered_df['DEALSIZE'] = filtered_df['DEALSIZE'].astype('category')
filtered_df['DEALSIZE_ENCODED'] = filtered_df['DEALSIZE'].cat.codes
print(filtered_df.PRODUCTLINE_ENCODED.unique())
#and dropping the original PRODUCTLINE column 
filtered_df = filtered_df.drop('PRODUCTLINE',axis=1)
filtered_df = filtered_df.drop('DEALSIZE',axis=1)
filtered_df.head()

['Motorcycles' 'Classic Cars' 'Trucks and Buses' 'Vintage Cars' 'Planes'
 'Ships' 'Trains']
[1 0 5 6 2 3 4]


,index,QUANTITYORDERED,PRICEEACH,SALES,STATUS,PRODUCTLINE_ENCODED,DEALSIZE_ENCODED
0,0,30,95.70,2871.00,Shipped,1,2
1,1,34,81.35,2765.90,Shipped,1,2
2,2,41,94.74,3884.34,Shipped,1,1
3,3,45,83.26,3746.70,Shipped,1,1
4,4,49,100.00,5205.27,Shipped,1,1


In [ ]:
#split the data into x and y 
X = filtered_df[['QUANTITYORDERED','PRICEEACH','SALES','PRODUCTLINE_ENCODED']]
print(X)
y = filtered_df[['DEALSIZE_ENCODED']]
print(y)

      QUANTITYORDERED  PRICEEACH    SALES  PRODUCTLINE_ENCODED
0                  30      95.70  2871.00                    1
1                  34      81.35  2765.90                    1
2                  41      94.74  3884.34                    1
3                  45      83.26  3746.70                    1
4                  49     100.00  5205.27                    1
...               ...        ...      ...                  ...
2612               40      55.69  2227.60                    3
2613               42      97.16  4080.72                    3
2614               20     100.00  2244.40                    3
2615               29     100.00  3978.51                    3
2616               34      62.24  2116.16                    3

[2617 rows x 4 columns]
      DEALSIZE_ENCODED
0                    2
1                    2
2                    1
3                    1
4                    1
...                ...
2612                 2
2613                 1
2614        

In [ ]:
#train test split, 80% training 20% testing , applying randomness to ensure reproducibility 
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2093, 4)
(524, 4)
(2093, 1)
(524, 1)


In [ ]:
clf = make_pipeline(StandardScaler(),SGDClassifier(max_iter=1000,tol=1e-3))

clf.fit(X_train,y_train.values.ravel())

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('sgdclassifier', SGDClassifier())])

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_pred,y_test))

0.9599236641221374


###97% accuracy in predicting the correct package size 


#Let's use PyCaret to see the performance metrics of the different classifiers that we can use. 

In [ ]:
from pycaret.classification import *

In [ ]:
df = pd.read_excel("/content/sales_data_sample.xlsx",usecols=['QUANTITYORDERED','PRICEEACH','SALES','STATUS','PRODUCTLINE','DEALSIZE'])
is_Shipped = df['STATUS'] == 'Shipped'
filtered_df = df[is_Shipped]
filtered_df = filtered_df.reset_index()
filtered_df['PRODUCTLINE'] = filtered_df['PRODUCTLINE'].astype('category')
filtered_df['PRODUCTLINE_ENCODED'] = filtered_df['PRODUCTLINE'].cat.codes
filtered_df['DEALSIZE'] = filtered_df['DEALSIZE'].astype('category')
filtered_df['DEALSIZE_ENCODED'] = filtered_df['DEALSIZE'].cat.codes
filtered_df = filtered_df.drop('PRODUCTLINE',axis=1)
filtered_df = filtered_df.drop('DEALSIZE',axis=1)


In [ ]:
data = filtered_df.sample(frac=0.80,random_state=786)
data_unseen = filtered_df.drop(data.index)
data.reset_index(inplace=True,drop=True)
data_unseen.reset_index(inplace=True,drop=True)
print('Data for modeling: '+str(data.shape))
print('Unseen data for predictions'+str(data_unseen.shape))

Data for modeling: (2094, 7)
Unseen data for predictions(523, 7)


In [ ]:
exp_clf = setup(data=data,target='DEALSIZE_ENCODED',session_id=123)

,Description,Value
0,session_id,123
1,Target,DEALSIZE_ENCODED
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(2094, 7)"
5,Missing Values,False
6,Numeric Features,5
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dt,Decision Tree Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.017
ada,Ada Boost Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.132
gbc,Gradient Boosting Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.619
lightgbm,Light Gradient Boosting Machine,0.9966,1.0000,0.9939,0.9967,0.9966,0.9938,0.9938,0.173
knn,K Neighbors Classifier,0.9918,0.9991,0.9869,0.9919,0.9918,0.9851,0.9851,0.121
et,Extra Trees Classifier,0.9884,0.9996,0.9690,0.9885,0.9882,0.9787,0.9789,0.486
lr,Logistic Regression,0.9850,0.9990,0.9895,0.9852,0.9850,0.9726,0.9728,0.923
lda,Linear Discriminant Analysis,0.9542,0.9940,0.9570,0.9561,0.9544,0.9172,0.9178,0.035
nb,Naive Bayes,0.8969,0.9798,0.9245,0.9156,0.9017,0.8189,0.8223,0.018
ridge,Ridge Classifier,0.8676,0.0000,0.6957,0.8846,0.8594,0.7503,0.7625,0.017
